In [1]:
import os
import itertools
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
from simulator_utilities import *
np.random.seed(7)

In [2]:
# ========================
# Preparation
# ========================
# Output
folder_path = r'/mnt/c/w/repos/slate_sim/notebook/data/single_gamma'
summary_file_path = os.path.join(folder_path, 'simulation_data_summary.csv')
context_file_path = os.path.join(folder_path, 'simulation_data_{0}.csv')

# Contexts 
contexts = {'platform': ['Mac', 'Windows'], 
            'network': ['wifi', 'wired'], 
            'country': ['US', 'CA']}
unique_contexts = [list(x) for x in itertools.product(*contexts.values())]

# Parameter statistics
params = {
          'x': {'mean': 1, 'min': 0, 'max': 4, 'std_range': [0.1, 1.1]},
          'y': {'mean': 1, 'min': 0, 'max': 3, 'std_range': [0.1, 1.1]}, 
          'z': {'mean': 1, 'min': 0, 'max': 2, 'std_range': [0.1, 1.1]}
         }

# Initialization
data = []
dist_context = {}
reward_range = [0.05, 0.35]
coefficient_range = [0.1, 1]
interaction2 = True
known_n = 20
descritization_policy = {'x': 4, 'y': 3, 'z': 2}
share_descritized_grid = True

# Confidence Interval statistics
ci_mean = 0
ci_std = 0.01
ci_dist = gen_distribution('normal', ci_mean, ci_std, 5000)[0]

# N for each unique context
if known_n:
    n_per_config = known_n
else:
    ci_width = 0.002
    ci_mult = 1.96
    n_per_config = int(((ci_mult*ci_std/(ci_width/2))**2//100+1)*100)
n_per_context = np.prod([x if isinstance(x, int) else len(x)for x in descritization_policy.values()]) * n_per_config
print('Data Size per Configuration: {:,}'.format(n_per_config))
print('Data Size per Context: {:,}'.format(n_per_context))
print('Total Data Size: {:,}'.format(n_per_context*len(unique_contexts)))

Data Size per Configuration: 20
Data Size per Context: 480
Total Data Size: 3,840


In [ ]:
# all_gamma_params = []

In [6]:
# ========================
# Generate data
# ========================
param_list = list(params.keys())
df_cols = list(contexts.keys()) + param_list + ['reward']
plot_pairs = [x for x in itertools.combinations(range(len(param_list)), 2)]
df_summary = pd.DataFrame()
df_all = pd.DataFrame()
coeff_all = [[0.12019595, 0.63518419, 0.96830226, 0.17708101, 0.18345325,0.8002412 ],
    [0.71518135, 0.18591909, 0.71323735, 0.47473797, 0.7525629 ,0.84263121],
    [0.90204983, 0.59199671, 0.33160712, 0.52421906, 0.65463157,0.37078185],
    [0.63522404, 0.33039906, 0.79644431, 0.94571491, 0.39494207, 0.12283538],
    [0.66273394, 0.22264627, 0.46584369, 0.41080309, 0.53285229,0.52803675],
    [0.82335403, 0.73190862, 0.26517287, 0.22790234, 0.61333563, 0.81023372],
    [0.15385043, 0.41863797, 0.15437206, 0.94464021, 0.12960767,0.567603  ],
    [0.73596655, 0.21766755, 0.43232531, 0.91792597, 0.63940162,0.80277422]]

gamma_param_all = [[0.12581686354841784, 0.9587723672104524, 0.5892252321817967],
[0.28456932884715136, 0.5794502415871884, 0.9530046583259941],
[0.21186398865555958, 0.8676770694338504, 0.7956440151232661],
[0.3327167704629427, 0.8020849264958928, 0.3534171495668579],
[0.6329803095169089, 0.8613506489399466, 0.5079810486234102],
[0.5244209894315414, 0.7256607308940456, 1.0952626583098724],
[0.8866024334344276, 0.2883860159703292, 0.7626985026756992],
[0.983667875614948, 0.20293592013287434, 0.12741018214844926]
]

if share_descritized_grid:
    for k, v in descritization_policy.items():
        if isinstance(v, int):
            descritization_policy[k] = [round(x,4) for x in np.linspace(params[k]['min'], params[k]['max'], v)]
            
for run in np.arange(1):
    for i, c in enumerate(unique_contexts):

        #display(Markdown('**[{0}/{1}] Generating data for context {2} ...**'.format(i+1, len(unique_contexts), c)))

        # [1] Generate Distributions
        n_dist = 5000
        dist_context[i], gamma_params = gen_param_reward(params, reward_range, n_dist, gamma_param_all[i])
        print(dist_context[i]["x"])
#         print(gamma_params)
        
#         saved_gamma_param = all_gamma_params[i]   # <--------- 
#         saved_gamma_param = None                # <--------- f
#         dist_context[i], gamma_params = gen_param_reward(params, reward_range, n_dist, saved_gamma_param)
#         all_gamma_params.append(gamma_params)   # <--------- f
        #plot_1d_param_reward(dist_context[i])

        # [2] Coefficients
        if interaction2:
            inter_terms = [x for x in itertools.combinations(range(len(param_list)), 2)]
        else:
            inter_terms = []
        n_coef = len(param_list) + len(inter_terms)
        #coefficients = np.random.uniform(coefficient_range[0], coefficient_range[1], n_coef)
        #coeff_all.append(coefficients)
        coefficients = coeff_all[i]

        # [3] Discretize parameter
        discretize_parameters(dist_context[i], descritization_policy, equal_distance=True)
        dist_context[i]['configs'] = gen_config_reward(dist_context[i], param_list)

        # [4] Reward formula
        reward_formula = param_list + ['{0}{1}'.format(param_list[x[0]], param_list[x[1]]) for x in inter_terms]
        dist_context[i]['configs']['reward_equation'] = formulate_equation(reward_formula, coefficients)
        dist_context[i]['configs']['config_rterms'] = add_interactions(dist_context[i]['configs']['config_rterms'], inter_terms)
        #display(Markdown('* {0}'.format(dist_context[i]['configs']['reward_equation'])))

        # [5] Generate data
        num_values = gen_data(dist_context[i], n_per_config, ci_dist, coefficients, reward_range)
        dist_context[i]['configs']['config_reward'] = np.array([x[-1] for x in num_values])
        c_data = [c + list(x) for x in num_values]
        data = data + c_data

        # [6] Plot 2D
        #plot_data = pd.DataFrame(num_values, columns=param_list + ['reward'])
        #plot_2d_paris(plot_data, param_list, inter_terms, round_to=0.05)

        # [7] To file
        df_context = pd.DataFrame(c_data, columns=df_cols)
        df_context = df_context.sample(frac=1)
        df_context['run'] = run
    #     df_context.to_csv(context_file_path.format(c), index=False)
        df_all = df_all.append(df_context)

        # [8] Summary
        df_mean = summarize_df(df_context, param_list, c)
        df_summary = df_summary.append(df_mean)

{'raw': array([1.44573175, 1.0221699 , 1.25747471, ..., 0.98263282, 0.9681965 ,
       0.87429107]), 'tick': array([0.60608112, 0.60676004, 0.60743896, ..., 3.99864216, 3.99932108,
       4.        ]), 'pdf': array([0.34942397, 0.34940837, 0.3493924 , ..., 0.35      , 0.35      ,
       0.35      ])}
{'raw': array([1.03907553, 1.04653148, 1.01195961, ..., 1.24221763, 0.99937709,
       0.93718984]), 'tick': array([0.25651953, 0.25726838, 0.25801722, ..., 3.99850231, 3.99925115,
       4.        ]), 'pdf': array([0.34944973, 0.34943646, 0.34942292, ..., 0.35      , 0.35      ,
       0.35      ])}
{'raw': array([1.07908174, 1.231582  , 1.1300331 , ..., 0.7912704 , 1.18290936,
       1.12114329]), 'tick': array([0.36694195, 0.36766871, 0.36839547, ..., 3.99854649, 3.99927324,
       4.        ]), 'pdf': array([0.34978737, 0.34978179, 0.34977608, ..., 0.35      , 0.35      ,
       0.35      ])}
{'raw': array([0.71580261, 0.80663552, 1.52758713, ..., 0.75704773, 0.7226937 ,
       1.33177

In [7]:
df_all.run.unique()

array([0])

In [8]:
out_path = os.path.join(folder_path, "df_all_4_3_2_50runs.csv")
df_all.to_csv(out_path)

In [ ]:
coeff_all

In [ ]:
coeff_all

In [ ]:
# 4-3-2 ground truth
min_actions = {}
min_reward = {}
grps_run = df_all.groupby(['run'])
for run in grps_run.groups.keys():
    df_temp = grps_run.get_group(run)
    grps = df_temp.groupby(['platform', 'network', 'country'])
    for grp in grps.groups.keys():
        if grp not in min_actions.keys():
            min_actions[grp] = {}
            min_reward[grp] = []
        temp_grp = grps.get_group(grp)
        temp_min = temp_grp['reward'].min()
        for action in ['x', 'y', 'z']:
            if action not in min_actions[grp].keys():
                min_actions[grp][action] = []
        min_actions[grp]['x'].append(df_temp[df_temp['reward'] == temp_min][['x','y','z']].drop_duplicates().values[:,0])
        min_actions[grp]['y'].append(df_temp[df_temp['reward'] == temp_min][['x','y','z']].drop_duplicates().values[:,1])
        min_actions[grp]['z'].append(df_temp[df_temp['reward'] == temp_min][['x','y','z']].drop_duplicates().values[:,2])
        min_reward[grp].append(temp_min)
        print(grp, temp_min, df_temp[df_temp['reward'] == temp_min][['x','y','z']].drop_duplicates().values)

In [ ]:
for key in min_actions.keys():
    print(key)
    print(np.mean(min_actions[key]['x']))
    print(np.mean(min_actions[key]['y']))
    print(np.mean(min_actions[key]['z']))

In [ ]:
min_reward